## PDI - Rebalancing data for February 2023

## Configuring packages

In [1]:
import pandas as pd
import numpy as np
import requests
import decouple
import sys 
sys.path.append('../')

from web3 import Web3
from abis import index_anatomy


infura_url = decouple.config("ETHEREUM_INFURA_URL")
web3_object = Web3(Web3.HTTPProvider(infura_url))

In [2]:
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

In [3]:
from defillama import DefiLlama
llama = DefiLlama()

In [4]:
import plotly.express as px
import plotly.graph_objects as go

In [5]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project must have a listing on Defi Llama.
    1. The project’s token should have been listed on CoinGecko with pricing data at least 1 year prior to the date of inclusion in the index. 
    1. The project should have a token that is native to Ethereum L1 or L2. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project. 
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. 
    1. Projects that have circular feedback characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
   <br>
1. **Pricing requirmeents**
    1. Token should have a manipulation resistant price feed in the form of a Chainlink price feed or a Uniswap V3 TWAP oracle with a cardinality of at least 300 and a minimum liquidity level of $1mm.
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The token should have in aggregate at least \$5mm of onchain liquidity across Uniswap v2, Uniswap v3, Sushiswap, Balancer v1, Balancer v2, Bancor v2 and Bancor v3.
    1. The price should experience no more than 4% price impact when executing a $100k trade.
    1. The token must have shown consistent DeFi liquidity on Ethereum.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

In [6]:
def normalize_symb(symb):
    return ''.join(c for c in symb if c.isalnum()).upper()

#### DefiLlama

In [7]:
protocols = llama.get_all_protocols()

In [8]:
top_defi_tokens = {} 
for pr in protocols[:1000]:
    if 'Ethereum' in pr['chains'] and pr['address'] and pr['address']!='-' and pr['symbol'] != '-':
        symbol = normalize_symb(pr['symbol']) 
        address = pr['address'].lower().split(":")[-1]
        top_defi_tokens[symbol] = {
            'symbol': symbol, 
            'address': address 
        }

# Switching NXM for WNXM

del top_defi_tokens['NXM']
top_defi_tokens['WNXM'] = {
    'symbol' : 'WNXM',
    'address' : '0x0d438f3b5175bebc262bf23753c1e53d03432bde'
}

#### CoinGecko

##### Tokens addresses

In [9]:
coins_list = pd.DataFrame(cg.get_coins_list(include_platform=True))

In [10]:
tokens = {} 
tokens_by_addr = {}
for index, coin in coins_list.iterrows():
    if len(coin['platforms']) >= 1 and 'ethereum' in list(coin['platforms'].keys()):
        symbol = normalize_symb(coin['symbol'])
        address = coin['platforms']['ethereum'].lower().split(":")[-1]
        token = {
            'symbol': symbol,
            'address': address 
        }
        tokens[symbol] = token
        tokens_by_addr[address] = token



In [11]:
# These token's addresses cant be found in the coingecko list of eth addresses
exclude_symbols = []
for symb, token in top_defi_tokens.items():
    if token['address'] not in tokens_by_addr.keys():
        exclude_symbols.append(token['symbol'])


In [12]:
for symb in exclude_symbols:
    del top_defi_tokens[symb]

top_defi_token_symbols = list(top_defi_tokens.keys())


##### Market data

In [13]:
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc')
for i in range(2, 20):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)
cg_tokens_data.set_index(cg_tokens_data['symbol'].str.upper(),inplace=True)

In [14]:
tokens_market_data = cg_tokens_data.set_index(cg_tokens_data['symbol'].str.upper())


In [15]:
duplicated = tokens_market_data.index[tokens_market_data.index.duplicated(keep='first')]
tokens_market_data = tokens_market_data.drop(index=duplicated)

In [16]:
tokens_market_data = tokens_market_data.rename(index={"ETH": "WETH"})
tokens_market_data.query('index in @top_defi_token_symbols',inplace=True)

### Token's supply check 

In [17]:

sp_passed_index = (tokens_market_data['circulating_supply'] / tokens_market_data['total_supply']) > 0.3
sp_passed = tokens_market_data[sp_passed_index]
        

### Retrieve historical data & livetime check

In [18]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for index, data in sp_passed[['id', 'symbol']].iterrows():
    id_ = data['id']
    symbol = data['symbol'].upper()
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', symbol])
    df_prices = df_prices[df_prices[symbol] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < 365:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < 365 days')
        exclude_list.append(symbol)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', symbol])
    df_mcaps = df_mcaps[df_mcaps[symbol] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < 365:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < 365 days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    

Excluding FRXETH, prices data available only for 47 < 365 days
Note: FLOKI, marketcap data available only for 322 < 365 days
Note: CFG, marketcap data available only for 138 < 365 days
Excluding EUL, prices data available only for 215 < 365 days
Excluding BTRFLY, prices data available only for 169 < 365 days
Excluding PSTAKE, prices data available only for 335 < 365 days
Note: DEUS, marketcap data available only for 332 < 365 days
Excluding BLUSD, prices data available only for 61 < 365 days
Excluding BRIDGE, prices data available only for 237 < 365 days
Excluding FLOOR, prices data available only for 330 < 365 days
Note: GAMMA, marketcap data available only for 350 < 365 days
Note: ORC, marketcap data available only for 352 < 365 days


### Non quantative checks

In [19]:
manual_exclusions = [
    {
        'symbol': 'WBTC',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'WETH',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'ERN',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'NFTX',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'RGT',
        'reason': 'Deprecated asset'
    },
    {
        'symbol': 'SUPER',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'BOND',
        'reason': 'Criteria 1.I'
    },
    {
        'symbol': 'USDC',
        'reason': 'stable'
    },
    {
        'symbol': 'FEI',
        'reason': 'stable'
    },
    {
        'symbol': 'SYN',
        'reason': 'bridge'
    },
    {
        'symbol': 'HBTC',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'BONE',
        'reason': 'Criteria 1.6'    
    },
    {
        'symbol': 'CRO',
        'reason': 'Not DeFi'
    }
    
    
]
for token in manual_exclusions:
    exclude_list.append(token['symbol'])

In [20]:
passed_q_and_non_q_checks = sp_passed.query('index not in @exclude_list' )


### Liquidity check

To derive a slippage threshold, consider an abstract univ2-like pool with $5m\$$ overall liquidity. 
I.e if pool has reserves $R_1$, $R_2$ and $p_1$ and $p_2$ are the correspondent tokens prices (in USD), then
$$R_1p_1 = R_2p_2 = 2.5m \$$$

Swapping $x$ token1 for token2 will result in the output amount $y$ such that
$$R_1\cdot R_2 = (R_1 + x) \cdot (R_2 - y)$$
$$y = \frac{R_2\cdot x}{R_1 + x}$$
Then the delivery price will be
$$p_d = \frac{y}{x} = \frac{R_2}{R_1 + x}$$
The spot price may be estimated as a delivery price for relatively small input amount $x_0$
$$p_s = \frac{y_0}{x_0} = \frac{R_2}{R_1 + x_0}$$

Then the theoretical slippage for this pool will be
$$slippage = \frac{p_d}{p_s} - 1 = \frac{R_1+x_0}{R_1+x} - 1 = \frac{p_1R_1+p_1x_0}{p_1R_1+p_1x}-1$$

In [21]:
liq = 5*10**6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5

In [22]:
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .5
max_slippage = round(2 * max_slippage, 2) / 2 
max_slippage

-0.04

In [23]:
url_0x = "https://api.0x.org/swap/v1/quote" 

In [24]:
decimals = 18
slippages = []
for symb, data in passed_q_and_non_q_checks.iterrows():
    try:
        query = {
            'buyToken': tokens[symb]['address'],
            'sellToken': 'ETH',
            'sellAmount': int(trade_value_tiny / float(cg_tokens_data[cg_tokens_data['symbol'] == 'eth']['current_price']) * 10 ** decimals),
            'enableSlippageProtection':'true'
        }
        
        # spot price is calculated as a price for 100$ swap
        resp = requests.get(url_0x, params=query)
        swap = resp.json()
        spot_price = float(swap['price'])
        
        query['sellAmount'] = int(trade_value / float(cg_tokens_data[cg_tokens_data['symbol'] == 'eth']['current_price']) * 10 ** decimals)
        resp = requests.get(url_0x, params=query)
        swap = resp.json()
        del_price = float(swap['price'])
        
        slippage = del_price / spot_price - 1
        slippages.append({
            'symb': symb,
            'spot_price': spot_price,
            'delivery_price': del_price,
            'slippage': slippage
        })
    except KeyError:
        continue
    

In [25]:
slippages_df = pd.DataFrame(slippages)
slippages_df.head(20)


,symb,spot_price,delivery_price,slippage
0,UNI,242.843075,242.760170,-0.000341
1,LDO,605.164555,602.250663,-0.004815
2,AAVE,18.886719,18.865242,-0.001137
3,FXS,151.448419,149.458026,-0.013142
4,CRV,1506.381486,1500.906516,-0.003635
5,RPL,41.622311,41.387759,-0.005635
6,MKR,2.304834,2.270113,-0.015065
7,SNX,637.392546,617.021496,-0.031960
8,1INCH,3012.004967,2921.455003,-0.030063
9,CVX,294.382514,292.394994,-0.006751


In [26]:
include_list = list(slippages_df[abs(slippages_df['slippage']) < abs(max_slippage)]['symb'])

liquidity_check = sp_passed.query('index in @include_list')

liquidity_check

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,6.670000,5024526855,20,6.665892e+09,1.821820e+08,6.930000,...,1.000000e+09,1.000000e+09,44.920000,-85.16950,2021-05-03T05:25:04.822Z,1.030000,546.64727,2020-09-17T01:20:38.214Z,None,2023-01-24T12:27:57.280Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,2.670000,2250575194,30,2.693030e+09,2.157099e+08,2.710000,...,1.000000e+09,1.000000e+09,7.300000,-63.18395,2021-08-20T08:35:20.158Z,0.406150,561.97968,2022-06-18T20:55:12.035Z,None,2023-01-24T12:27:55.918Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,85.940000,1221122015,45,1.372330e+09,1.190398e+08,88.410000,...,1.600000e+07,1.600000e+07,661.690000,-87.02625,2021-05-18T21:19:59.514Z,26.020000,229.88338,2020-11-05T09:20:11.928Z,None,2023-01-24T12:27:43.203Z
FXS,frax-share,fxs,Frax Share,https://assets.coingecko.com/coins/images/1342...,10.710000,785089500,58,1.069819e+09,4.275047e+07,11.140000,...,9.982298e+07,9.982298e+07,42.800000,-74.97302,2022-01-12T15:22:27.465Z,1.500000,612.59826,2021-06-25T16:50:51.447Z,None,2023-01-24T12:27:59.027Z
CRV,curve-dao-token,crv,Curve DAO,https://assets.coingecko.com/coins/images/1212...,1.079000,780592818,60,3.564749e+09,1.371266e+08,1.110000,...,1.889957e+09,3.303030e+09,15.370000,-92.97868,2020-08-14T00:00:00.000Z,0.331577,225.51227,2020-11-05T13:09:50.181Z,None,2023-01-24T12:27:38.576Z
RPL,rocket-pool,rpl,Rocket Pool,https://assets.coingecko.com/coins/images/2090...,38.760000,743324488,62,7.433245e+08,3.075980e+07,41.060000,...,1.911341e+07,1.911341e+07,59.460000,-34.58975,2021-11-09T04:03:31.003Z,0.008847,439477.47126,2018-08-28T00:00:00.000Z,None,2023-01-24T12:27:41.775Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,700.220000,631834590,71,7.049269e+08,2.137832e+07,713.840000,...,9.776310e+05,1.005577e+06,6292.310000,-88.85910,2021-05-03T21:54:29.333Z,168.360000,316.38780,2020-03-16T20:52:36.527Z,None,2023-01-24T12:27:59.107Z
SNX,havven,snx,Synthetix Network,https://assets.coingecko.com/coins/images/3406...,2.540000,611350885,73,7.944541e+08,3.342400e+07,2.620000,...,3.125248e+08,3.125248e+08,28.530000,-91.09004,2021-02-14T01:12:38.505Z,0.034786,7208.07443,2019-01-06T00:00:00.000Z,"{'times': 4.08514372678724, 'currency': 'usd',...",2023-01-24T12:28:00.520Z
1INCH,1inch,1inch,1inch,https://assets.coingecko.com/coins/images/1346...,0.530156,423985136,99,7.977554e+08,3.614553e+07,0.548295,...,1.500000e+09,1.500000e+09,8.650000,-93.85685,2021-10-27T08:24:54.808Z,0.373690,42.22555,2022-12-30T12:35:04.113Z,None,2023-01-24T12:27:49.280Z


In [27]:
qualified_assets = liquidity_check.query('index not in @exclude_list')

qualified_assets

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,6.670000,5024526855,20,6.665892e+09,1.821820e+08,6.930000,...,1.000000e+09,1.000000e+09,44.920000,-85.16950,2021-05-03T05:25:04.822Z,1.030000,546.64727,2020-09-17T01:20:38.214Z,None,2023-01-24T12:27:57.280Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,2.670000,2250575194,30,2.693030e+09,2.157099e+08,2.710000,...,1.000000e+09,1.000000e+09,7.300000,-63.18395,2021-08-20T08:35:20.158Z,0.406150,561.97968,2022-06-18T20:55:12.035Z,None,2023-01-24T12:27:55.918Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,85.940000,1221122015,45,1.372330e+09,1.190398e+08,88.410000,...,1.600000e+07,1.600000e+07,661.690000,-87.02625,2021-05-18T21:19:59.514Z,26.020000,229.88338,2020-11-05T09:20:11.928Z,None,2023-01-24T12:27:43.203Z
FXS,frax-share,fxs,Frax Share,https://assets.coingecko.com/coins/images/1342...,10.710000,785089500,58,1.069819e+09,4.275047e+07,11.140000,...,9.982298e+07,9.982298e+07,42.800000,-74.97302,2022-01-12T15:22:27.465Z,1.500000,612.59826,2021-06-25T16:50:51.447Z,None,2023-01-24T12:27:59.027Z
CRV,curve-dao-token,crv,Curve DAO,https://assets.coingecko.com/coins/images/1212...,1.079000,780592818,60,3.564749e+09,1.371266e+08,1.110000,...,1.889957e+09,3.303030e+09,15.370000,-92.97868,2020-08-14T00:00:00.000Z,0.331577,225.51227,2020-11-05T13:09:50.181Z,None,2023-01-24T12:27:38.576Z
RPL,rocket-pool,rpl,Rocket Pool,https://assets.coingecko.com/coins/images/2090...,38.760000,743324488,62,7.433245e+08,3.075980e+07,41.060000,...,1.911341e+07,1.911341e+07,59.460000,-34.58975,2021-11-09T04:03:31.003Z,0.008847,439477.47126,2018-08-28T00:00:00.000Z,None,2023-01-24T12:27:41.775Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,700.220000,631834590,71,7.049269e+08,2.137832e+07,713.840000,...,9.776310e+05,1.005577e+06,6292.310000,-88.85910,2021-05-03T21:54:29.333Z,168.360000,316.38780,2020-03-16T20:52:36.527Z,None,2023-01-24T12:27:59.107Z
SNX,havven,snx,Synthetix Network,https://assets.coingecko.com/coins/images/3406...,2.540000,611350885,73,7.944541e+08,3.342400e+07,2.620000,...,3.125248e+08,3.125248e+08,28.530000,-91.09004,2021-02-14T01:12:38.505Z,0.034786,7208.07443,2019-01-06T00:00:00.000Z,"{'times': 4.08514372678724, 'currency': 'usd',...",2023-01-24T12:28:00.520Z
1INCH,1inch,1inch,1inch,https://assets.coingecko.com/coins/images/1346...,0.530156,423985136,99,7.977554e+08,3.614553e+07,0.548295,...,1.500000e+09,1.500000e+09,8.650000,-93.85685,2021-10-27T08:24:54.808Z,0.373690,42.22555,2022-12-30T12:35:04.113Z,None,2023-01-24T12:27:49.280Z


### Marketcap ranking & filtering

In [28]:
chosen_tokens = qualified_assets.sort_values("market_cap", ascending=False)

In [29]:
chosen_tokens_symbols = list(chosen_tokens.index)
prices_data = prices_data[chosen_tokens_symbols]
marketcaps = marketcaps[chosen_tokens_symbols]

In [30]:
without_nan_index = (marketcaps.isnull().sum(axis=1) == 0) & (prices_data.isnull().sum(axis=1) == 0)
marketcaps = marketcaps[without_nan_index]
prices_data = prices_data[without_nan_index]

## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 30%.
* All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
* Any asset with a weight below 1% will be removed from the index.


### Weight mcaps

In [31]:
weights = marketcaps.div(marketcaps.sum(axis=1), axis=0)
weights = weights.sort_values(weights.last_valid_index(), axis=1, ascending=False)

In [32]:
fig = px.line(weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    ## showlegend=False,
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights without max constraint'
)

### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [33]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum(axis=1)
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum(axis=1), axis=0).mul(c, axis=0)
    
    return w

In [34]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum(axis=1), axis=0)
    
    return w

In [35]:
adjusted_weights = adjust_weights(weights, max_weight=0.3) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=0.01)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=0.3)

adjusted_weights = adjusted_weights.sort_values(adjusted_weights.last_valid_index(),ascending=False, axis = 1)
# adjusted_weights

In [36]:
fig = px.line(adjusted_weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights with max constraint'
)

### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [37]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<76]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [38]:
last_weights = adjusted_weights.iloc[-1].dropna()

In [48]:
converted_last_weights = convert_weights(last_weights)
converted_last_weights


UNI       76
LDO       41
AAVE      27
CRV       17
FXS       16
RPL       15
MKR       14
SNX       14
1INCH      9
CVX        8
BAL        5
SUSHI      5
YFI        5
RBN        3
SPELL      0
BNT        0
BADGER     0
TEMPLE     0
WNXM       0
ALCX       0
Name: 2023-01-23 00:00:00, dtype: int64

## Summary table

In [40]:
pdi = pd.DataFrame()
pdi.index = chosen_tokens.index
pdi['name'] = chosen_tokens['name']
pdi['market_cap'] = marketcaps.iloc[-1].astype(int)
pdi['price'] = prices_data.iloc[-1]
pdi['weight'] = last_weights
pdi['weight_converted'] = converted_last_weights
pdi['address'] = [top_defi_tokens[symb]['address'] for symb in pdi.index]

pdi = pdi[pdi['weight'] > 0]
pdi = pdi.sort_values("market_cap",ascending=False)

In [46]:
#Querying current contract to ascertain which assets will be dropped

pdi_token_address = '0x632806BF5c8f062932Dd121244c9fbe7becb8B48'
ia_contract = web3_object.eth.contract(address=pdi_token_address,abi=index_anatomy)
addresses, weights = ia_contract.functions.anatomy().call()

for address in addresses:
    address = address.lower()
    if address not in list(pdi['address']):
        symbol = tokens_by_addr[address]['symbol']
        data = cg_tokens_data.loc[symbol]
        pdi.loc[symbol] = [data['name'],data['market_cap'],data['current_price'],0,0,address]
pdi

,name,market_cap,price,weight,weight_converted,address
symbol,,,,,,
UNI,Uniswap,5211773831,6.908366,0.300000,76,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
LDO,Lido DAO,1852088328,2.217777,0.157963,41,0x5a98fcbea516cf06857215779fd812ca3bef1b32
AAVE,Aave,1219104014,85.546401,0.103976,27,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9
CRV,Curve DAO,777610897,1.075760,0.066322,17,0xd533a949740bb3306d119cc777fa900ba034cd52
FXS,Frax Share,734140834,10.072906,0.062614,16,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0
RPL,Rocket Pool,659819920,34.514341,0.056276,15,0xd33526068d116ce69f19a9ee46f0bd304f21a51f
MKR,Maker,640957875,710.980853,0.054667,14,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2
SNX,Synthetix Network,624002438,2.596573,0.053221,14,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f
1INCH,1inch,411242252,0.515531,0.035075,9,0x111111111117dc0aa78b770fa6a738034120c302


In [42]:
zipped_assets = list(zip(list(pdi['address']),list(pdi['weight_converted'])))
sorted_assets = sorted(zipped_assets,key = lambda x: int(x[0],base=0) )

asset_string = ''
for i in sorted_assets:
    asset_string += f'{i[0]},'
asset_string = asset_string[:-1]
print(asset_string)
print([x[1] for x in sorted_assets])

0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,0x111111111117dc0aa78b770fa6a738034120c302,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,0x5a98fcbea516cf06857215779fd812ca3bef1b32,0x6123b0049f904d730db3c36a31167d9d4121fa6b,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,0xba100000625a3754423978a60c9317c58a424e3d,0xbbbbca6a901c926f240b89eacb641d8aec7aeafd,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,0xd33526068d116ce69f19a9ee46f0bd304f21a51f,0xd533a949740bb3306d119cc777fa900ba034cd52
[5, 9, 76, 16, 8, 41, 3, 5, 27, 14, 5, 0, 14, 15, 17]
